In [3]:
from dotenv import load_dotenv
load_dotenv()

# .env 파일 강제 로드
load_dotenv(override=True)

True

In [5]:
import pandas as pd
adv_df = pd.read_csv('./의약품안전사용서비스(DUR)_병용금기 품목리스트 2024.5.csv',  encoding='cp949')

C:\Users\playdata2\AppData\Local\Temp\ipykernel_2668\3761064063.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  adv_df = pd.read_csv('./의약품안전사용서비스(DUR)_병용금기 품목리스트 2024.5.csv',  encoding='cp949')


In [6]:
adv_df.head()

,성분명A,성분코드A,제품코드A,제품명A,업소명A,구분A,성분명B,성분코드B,제품코드B,제품명B,업소명B,구분B,고시번호,고시적용일,금기사유,비고
0,aceclofenac,100901ACH,648101510,에이서캡슐(아세클로페낙)_(0.1g/1캡슐),경동제약(주),급여,ketorolac tromethamine,180001ATB,653400910,케토라신정(케토롤락트로메타민)_(10mg/1정),동국제약(주),급여,20050017,2005-03-09,중증의 위장관계 이상반응,NaN
1,aceclofenac,100901ACH,648101510,에이서캡슐(아세클로페낙)_(0.1g/1캡슐),경동제약(주),급여,ketorolac tromethamine,180033BIJ,57200121,타라신주(케토롤락트로메타민)_(30mg/1mL),지피테라퓨틱스코리아주식회사,급여,20050017,2005-03-09,중증의 위장관계 이상반응,NaN
2,aceclofenac,100901ACH,648101510,에이서캡슐(아세클로페낙)_(0.1g/1캡슐),경동제약(주),급여,ketorolac tromethamine,180033BIJ,59000681,케롤락주(케토롤락트로메타민염)_(30mg/1mL),주식회사케이에스제약,급여,20050017,2005-03-09,중증의 위장관계 이상반응,NaN
3,aceclofenac,100901ACH,648101510,에이서캡슐(아세클로페낙)_(0.1g/1캡슐),경동제약(주),급여,ketorolac tromethamine,180033BIJ,645902651,케로라주(케토롤락트로메타민염)_(30mg/1mL),동광제약(주),급여,20050017,2005-03-09,중증의 위장관계 이상반응,NaN
4,aceclofenac,100901ACH,648101510,에이서캡슐(아세클로페낙)_(0.1g/1캡슐),경동제약(주),급여,ketorolac tromethamine,180033BIJ,646002281,타벨주(케토롤락트로메타민염)_(30mg/1mL),(주)메디카코리아,급여,20050017,2005-03-09,중증의 위장관계 이상반응,NaN


In [8]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./의약품안전사용서비스(DUR)_병용금기 품목리스트 2024.5.csv', encoding='cp949')
documents = loader.load()
print(len(documents))
documents[0]

791230


Document(metadata={'source': './의약품안전사용서비스(DUR)_병용금기 품목리스트 2024.5.csv', 'row': 0}, page_content='성분명A: aceclofenac\n성분코드A: 100901ACH\n제품코드A: 648101510\n제품명A: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)\n업소명A: 경동제약(주)\n구분A: 급여\n성분명B: ketorolac tromethamine\n성분코드B: 180001ATB\n제품코드B: 653400910\n제품명B: 케토라신정(케토롤락트로메타민)_(10mg/1정)\n업소명B: 동국제약(주)\n구분B: 급여\n고시번호: 20050017\n고시적용일: 2005-03-09\n금기사유: 중증의 위장관계 이상반응\n비고: ')

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [4]:
import os
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

print(PINECONE_INDEX_NAME)
print(PINECONE_NAMESPACE)
print(PINECONE_API_KEY)

241218-ai-medicine
241218-ai-medicine-ns
pcsk_7GsZj2_NJyHz8wTEQ8Q19KUXDBqWjTLqbzHkvYPcyKieA2yUACz7jMcZ2QwxD34QPdQx5S


In [12]:
#Pinecone 클라이언트
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

def batch_documents(documents, batch_size):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

batch_size = 5000  # 배치 크기 설정
for batch in batch_documents(documents, batch_size):
    PineconeVectorStore.from_documents(
        batch,
        embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key=PINECONE_API_KEY
    )

# vector_db = PineconeVectorStore.from_documents(
#     documents,
#     embeddings,
#     index_name=PINECONE_INDEX_NAME,
#     namespace=PINECONE_NAMESPACE,
#     pinecone_api_key=PINECONE_API_KEY
# )


PineconeApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 18 Dec 2024 06:03:07 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1287', 'x-pinecone-request-id': '6339427293384721043', 'x-envoy-upstream-service-time': '3', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached your write unit limit for the current month (2000000). To continue writing data, upgrade your plan.","details":[]}


In [13]:
import pinecone
index = pinecone.Index(PINECONE_INDEX_NAME)

# 네임스페이스 내 모든 데이터 삭제
index.delete(delete_all=True, namespace=PINECONE_NAMESPACE)

TypeError: Index.__init__() missing 1 required positional argument: 'host'

In [ ]:
import pinecone
import os

# Pinecone 초기화
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),        # 환경 변수에서 API 키 불러오기
    environment=os.getenv("PINECONE_ENVIRONMENT")  # 환경 변수에서 환경 설정 불러오기
)

# 인덱스 객체 생성
index = pinecone.Index(os.getenv("PINECONE_INDEX_NAME"))

# 네임스페이스 내 모든 데이터 삭제
index.delete(delete_all=True, namespace=os.getenv("PINECONE_NAMESPACE"))

print("네임스페이스 내 데이터가 성공적으로 삭제되었습니다.")

In [8]:
# 클라이언트 객체
from langchain_pinecone import PineconeVectorStore
vector_db = PineconeVectorStore(
    embedding=embeddings,
    index_name=PINECONE_INDEX_NAME,
    namespace=PINECONE_NAMESPACE,
    pinecone_api_key=PINECONE_API_KEY)

results = vector_db.similarity_search(
    "에이서캡슐(아세클로페낙)_(0.1g/1캡슐)",
    k=5,
    namespace=PINECONE_NAMESPACE) # 검색의 논리적인 단위
results

[Document(id='48b37556-926d-4117-b1b7-d5d83a21e6d5', metadata={}, page_content="{'제품명': '아세틸란캡슐(아세틸시스테인)', '제품영문명': 'Acetylan Cap.', '업체명': '지엘파마(주)', '업체영문명': 'GL Pharma', '품목구분': '의약품', '주성분': '아세틸시스테인', '첨가제': '라우릴황산나트륨,스테아르산마그네슘,상,하부 흰색의 경질캡슐,옥수수전분', '전문의약품': '일반의약품', '완제/원료': '완제의약품', '마약구분': nan, '모양': '장방형', '색상': nan, '제형': nan, '장축': 18.8, '단축': 6.3, '묶음의약품정보': nan, 'e은약요': 201901857.0, '주성분영문': 'Acetylcysteine'}"),
 Document(id='e2bffbd0-9a6c-461e-ba94-14ae21d7859f', metadata={}, page_content="{'제품명': '아제인캡슐(아세틸시스테인)', '제품영문명': 'Azein Cap.', '업체명': '알파제약(주)', '업체영문명': 'Alpha Pharm. Co., Ltd.', '품목구분': '의약품', '주성분': '아세틸시스테인', '첨가제': '유당수화물,(상ㆍ하)청색캡슐,스테아르산마그네슘,탤크,실리콘디옥시드,옥수수전분,(상부)백색(하부)청색캡슐', '전문의약품': '일반의약품', '완제/원료': '완제의약품', '마약구분': nan, '모양': '장방형', '색상': nan, '제형': nan, '장축': 19.1, '단축': 6.63, '묶음의약품정보': nan, 'e은약요': 198902058.0, '주성분영문': 'Acetylcysteine'}"),
 Document(id='c5d3635f-123d-4c7b-b40b-2f36d925eb63', metadata={}, page_content="{'제품명': '아시틴캡슐(아세틸시스테인)', '제품영문명'